In [ ]:
from typing import Any
import yaml

In [ ]:
def sweep_constructor(
    loader: yaml.SafeLoader, node: yaml.nodes.MappingNode
) -> dict[str, Any]:
    values = loader.construct_sequence(node.value[0][1])
    return {"type": "sweep", "values": values}


def coupled_sweep_constructor(
    loader: yaml.SafeLoader, node: yaml.nodes.MappingNode
) -> dict[str, Any]:
    target = node.value[0][1].value
    values = loader.construct_sequence(node.value[1][1])
    return {"target": target, "values": values, "type": "coupled-sweep"}


def get_loader():
    loader = yaml.SafeLoader
    loader.add_constructor("!sweep", sweep_constructor)
    loader.add_constructor("!coupled-sweep", coupled_sweep_constructor)
    return loader

In [ ]:
cfg = yaml.load(open("./config.yaml"), Loader=get_loader())

In [ ]:
cfg

In [ ]:
class Run:
    def __init__(self, config: dict):
        self.config = config
        self.run_configs = []

    def _cfg_helper(self, cfg_node: dict) -> Tuple[list[Any], list[Any]]
        sweep_targets = []
        coupled_targets = []

        for key, node in cfg_node.items():
            if isinstance(node, dict) and "type" in node:
                if node["type"] == "sweep":
                    sweep_targets.append((key, node))
                elif node["type"] == "coupled-sweep":
                    coupled_targets.append((key, node))
            elif isinstance(node, dict):
                self._cfg_helper(key, node)
            else:
                continue  # we ignore leafs

        return sweep_targets, coupled_targets

    def _construct_run_configs(self):
        sweep_targets, coupled_targets = self._cfg_helper(self.config)

        # make numpy structured_array or list of named tuples
        num_elements = 1
        dtype = []
        for swt in sweep_targets: 
            num_elements *= len(swt["values"])
            dtype.append((swt[0], "O")) # make sure the types are set 

        for cwt in coupled_targets:
            num_elements *= len(cwt["values"])

        # copy config n times 
        cfgs = [self.config for _ in range(num_elements)]

        # go over structured array and make a new config for each element

    